In [6]:
from sys import path

path.append(r"/opt/configs/dev")

from config import MONGO_DB, MONGO_HOST, MONGO_PASSWD, MONGO_USER, MONGO_PORT


from pymongo import MongoClient


cli = MongoClient(f"mongodb://{MONGO_USER}:{MONGO_PASSWD}@{MONGO_HOST}:{MONGO_PORT}")
db = cli['leaked']

数据格式很乱，不是标准 JSON。
需要逐个文件处理，用正则提取有效信息。

## address_merge_with_mobile_data.json

```
{"_source":{"APP_TYPE":"1229999","ATTR_IDENT_LABEL":"07","ATTR_IDENT_LABEL_DETL":"{"LABEL_DETL":[{"07":"shga_wa.ods_nb_tab_goods"}]}","BIG_SOURCE":"WA","COUNT":"3","DATA_SOURCE":"115","DETAIL":"{"nameinfo":[{"name":"刘婵女士"}]}","FIRST_TIME":"1562126476","IDENTITY_TYPE":"mobile","IDENTITY_VALUE":"18627855112","LAST_TIME":"1562557677","MRG_ID":"cda7c53c06248628cd33777dff71d9a3","SRC_ADDRESS":"智慧e谷大楼(5楼)","SRC_ID":"cda7c53c06248628cd33777dff71d9a3","TABLE_SOURCE":"shga_wa.ods_nb_tab_goods"},"_type":"a","sort":[33150]}"""
```

In [22]:
import re

raw = """{"_source":{"APP_TYPE":"1229999","ATTR_IDENT_LABEL":"07","ATTR_IDENT_LABEL_DETL":"{"LABEL_DETL":[{"07":"shga_wa.ods_nb_tab_goods"}]}","BIG_SOURCE":"WA","COUNT":"3","DATA_SOURCE":"115","DETAIL":"{"nameinfo":[{"name":"刘婵女士"}]}","FIRST_TIME":"1562126476","IDENTITY_TYPE":"mobile","IDENTITY_VALUE":"18627855112","LAST_TIME":"1562557677","MRG_ID":"cda7c53c06248628cd33777dff71d9a3","SRC_ADDRESS":"智慧e谷大楼(5楼)","SRC_ID":"cda7c53c06248628cd33777dff71d9a3","TABLE_SOURCE":"shga_wa.ods_nb_tab_goods"},"_type":"a","sort":[33150]}"""
regex = r"\"name\":\"(?P<name>[^\"]+)\".*?\"IDENTITY_TYPE\":\"(?P<type>[^\"]+)\".*?\"IDENTITY_VALUE\":\"(?P<val>[^\"]+)\".*?\"SRC_ADDRESS\":\"(?P<addr>[^\"]+)\".*?"

addr_regex = re.compile(regex)
addr_regex.findall(raw)


[('刘婵女士', 'mobile', '18627855112', '智慧e谷大楼(5楼)')]

In [26]:
import codecs

# {"_source":{"APP_TYPE":"1229999","ATTR_IDENT_LABEL":"07","ATTR_IDENT_LABEL_DETL":"{"LABEL_DETL":[{"07":"shga_wa.ods_nb_tab_goods"}]}","BIG_SOURCE":"WA","COUNT":"3","DATA_SOURCE":"115","DETAIL":"{"nameinfo":[{"name":"刘婵女士"}]}","FIRST_TIME":"1562126476","IDENTITY_TYPE":"mobile","IDENTITY_VALUE":"18627855112","LAST_TIME":"1562557677","MRG_ID":"cda7c53c06248628cd33777dff71d9a3","SRC_ADDRESS":"智慧e谷大楼(5楼)","SRC_ID":"cda7c53c06248628cd33777dff71d9a3","TABLE_SOURCE":"shga_wa.ods_nb_tab_goods"},"_type":"a","sort":[33150]}

col = db['persons']
i = 0
with codecs.open(r"/home/laisky/Downloads/address_merge_with_mobile_data.json", 'r', 'utf8') as fp:
    for line in fp:
        matched = addr_regex.findall(line)
        if len(matched) != 1 or matched[0][1].lower() != "mobile":
            continue
            
        docu = {
            "name": matched[0][0],
            "mobile": matched[0][2],
            "address": matched[0][3],
        }
        col.update_one(
            filter={"mobile": docu['mobile']},
            update={"$set": docu},
            upsert=True,
        )
        i += 1
        if i % 100 == 0:
            print(f"inserted {i}")

## case_data_index.json

```
{"_id":"AW8713gL3Lv7g0L99dUe","_index":"nb_theme_address_case_dwd-total","_score":null,"_source":{"ADDR_DETL":"{"caseState":"受理","policeJson":{"bm":"310113630000","cjdwmc":"上海市公安局宝山分局顾村派出所","cjdwdh":null},"caseAddress":"上海市宝山区教育路555弄12号501室","tableSource":"shga_dwd.base_wsba_hx_a_ajjbqk_df"}","ADDR_TYPE":"02","BUILDING_L9":"12号","CASE_NAME":"教育路555弄12号501室纠纷案","CASE_NUMBER":"C3101136300002013100579","CITY_L2":"上海市","CONFIDENCE":"4","COUNTY_L3":"宝山区","GEOHASH7":"wtw66kj","GEOHASH8":"wtw66kjt","GEOHASH9":"wtw66kjt8","LATITUDE":"31.355935","LOC_SOURCE":"0","LONGITUDE":"121.395524","MCS_ID":"4622821d1ccb293cbad6e029dc9a5aae","NONG_L6":"555弄","PROVINCE_L1":"上海市","ROAD_L5":"教育路","ROOM_L12":"501室","STD_ADDRESS":"上海市上海市宝山区教育路555弄12号501室"},"_type":"a","sort":[26100]}

```

## person_info.json

```
{"_id":"AXtamfr9mkHrMSbQCSme","_index":"person_address_label_info_master","_score":null,"_source":{"AGE":6,"BIRTHDAY":"2015","BPLACE":"浙江省衢州市柯城区","HHPLACE":"浙江省衢州市柯城区九华乡上铺村徐莫39号","IDNO":"330802201501207944","IDTYPE":"01","PHOTO":"{"身份证":["http://oss-cn-shanghai-shga-d01-a.ops.ga.sh/shga-ryzp/CSJ/ZHEJIANG_CZRK_ZP/59578841_330802.jpg"]}","QUERY_STRING":"  浙江省衢州市柯城区  浙江省衢州市柯城区九华乡上铺村徐莫39号 6 15 2015 ","RNAME":"危婧彤","SEX":"女"},"_type":"a","sort":[6754953]}
{"_id":"AXtamMSkxRtHEfZelWKn","_index":"person_address_label_info_master","_score":null,"_source":{"AGE":43,"BIRTHDAY":"1978","BPLACE":"湖北省十堰市张湾区","HEIGHT":"160","IDNO":"420303197812271726","IDTYPE":"01","QUERY_STRING":"  湖北省十堰市张湾区   43 78 1978 ","RNAME":"王立华","SEX":"女"},"_type":"a","sort":[6754953]}

```

In [30]:
raw = """{"_id":"AXtamfr9mkHrMSbQCSme","_index":"person_address_label_info_master","_score":null,"_source":{"AGE":6,"BIRTHDAY":"2015","BPLACE":"浙江省衢州市柯城区","HHPLACE":"浙江省衢州市柯城区九华乡上铺村徐莫39号","IDNO":"330802201501207944","IDTYPE":"01","PHOTO":"{"身份证":["http://oss-cn-shanghai-shga-d01-a.ops.ga.sh/shga-ryzp/CSJ/ZHEJIANG_CZRK_ZP/59578841_330802.jpg"]}","QUERY_STRING":"  浙江省衢州市柯城区  浙江省衢州市柯城区九华乡上铺村徐莫39号 6 15 2015 ","RNAME":"危婧彤","SEX":"女"},"_type":"a","sort":[6754953]}
"""
regex = r"\"IDNO\":\"([^\"]+)\".*?\"QUERY_STRING\":\"([^\"]+)\".*?\"RNAME\":\"([^\"]+)\".*?\"SEX\":\"([^\"]+)\".*?"

person_regex = re.compile(regex)
person_regex.findall(raw)

json.loads(raw)

JSONDecodeError: Expecting ',' delimiter: line 1 column 233 (char 232)

In [ ]:
col = db['persons']
i = 0
with codecs.open(r"/home/laisky/Downloads/address_merge_with_mobile_data.json", 'r', 'utf8') as fp:
    for line in fp:
        matched = addr_regex.findall(line)
        if len(matched) != 1 or matched[0][1].lower() != "mobile":
            continue
            
        docu = {
            "name": matched[0][0],
            "mobile": matched[0][2],
            "address": matched[0][3],
        }
        col.update_one(
            filter={"mobile": docu['mobile']},
            update={"$set": docu},
            upsert=True,
        )
        i += 1
        if i % 100 == 0:
            print(f"inserted {i}")